In [71]:
import numpy as np
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.mappers import JordanWignerMapper

Hamiltoniano con las matrices one-body y two-body para V0 = 1

In [72]:
# hpq = np.array([
#     [ 4.9348022,  0.       ,  0.       ,  0.       ],
#     [ 0.       ,  4.9348022,  0.       ,  0.       ],
#     [ 0.       ,  0.       , 19.7392088,  0.       ],
#     [ 0.       ,  0.       ,  0.       , 19.7392088]
# ])

hpq = np.array([
    [ 4.9348022,  0.       ],
    [ 0.       , 19.7392088]
])

In [73]:
# hpqrs = np.array([[[
#          [1.5, 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 1. , 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [1.5, 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 1. , 0. ]],

#         [[0. , 0. , 1. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [1. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 0. , 1. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [1. , 0. , 0. , 0. ]]],


#        [[[0. , 1.5, 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 1.5, 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1. ]],

#         [[0. , 0. , 0. , 1. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 1. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 1. , 0. , 0. ]]],


#        [[[0. , 0. , 1. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [1. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 0. , 1. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [1. , 0. , 0. , 0. ]],

#         [[1. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 1.5, 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [1. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 1.5, 0. ]]],


#        [[[0. , 0. , 0. , 1. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 1. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 1. , 0. , 0. ]],

#         [[0. , 1. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1.5],
#          [0. , 0. , 0. , 0. ]],

#         [[0. , 0. , 0. , 0. ],
#          [0. , 1. , 0. , 0. ],
#          [0. , 0. , 0. , 0. ],
#          [0. , 0. , 0. , 1.5]]]])

hpqrs = np.array([[[
         [1.5, 0. ],
         [0. , 1. ]],

        [[0. , 1. ],
         [1. , 0. ]]],


       [[[0. , 1. ],
         [1. , 0. ]],

        [[1. , 0. ],
         [0. , 1.5]]]])

In [74]:
h1_a = np.array(hpq, dtype=float)
h2_aa = np.array(hpqrs, dtype=float)
hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa)

# Pasamos el hamiltoniano a terminos de operadores de qubit con el mapper
mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

In [75]:
qubit_op.paulis

PauliList(['IIII', 'IIZI', 'IIIZ', 'IZII', 'IZIZ', 'YYYY', 'XXYY',
           'YYXX', 'XXXX', 'ZIII', 'ZIIZ', 'IZZI', 'ZIZI'])

In [76]:
qubit_op.coeffs

array([ 25.924011 +0.j, -10.4946044+0.j,  -3.0924011+0.j,  -3.0924011+0.j,
         0.375    +0.j,   0.25     +0.j,   0.25     +0.j,   0.25     +0.j,
         0.25     +0.j, -10.4946044+0.j,   0.25     +0.j,   0.25     +0.j,
         0.375    +0.j])

In [77]:
hamiltonian

Hamiltoniano de todo el sistema para V0 = 1

In [78]:
from qiskit.quantum_info import Operator, SparsePauliOp
import numpy as np

# Definir la matriz de Hamiltoniano
H = np.array([
    [54.34802201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 32.1088132, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 32.1088132, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 11.3696044, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 46.9132198, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 24.674011, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 25.674011, 0, 0, -1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4.9348022, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 46.9132198, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, -1, 0, 0, 25.674011, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24.674011, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.9348022, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 40.9784176, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19.7392088, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19.7392088, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])

# Crear un operador a partir de la matriz Hamiltoniana
operator = Operator(H)

pauli_op = SparsePauliOp.from_operator(operator)

In [79]:
qubit_op.paulis

PauliList(['IIII', 'IIZI', 'IIIZ', 'IZII', 'IZIZ', 'YYYY', 'XXYY',
           'YYXX', 'XXXX', 'ZIII', 'ZIIZ', 'IZZI', 'ZIZI'])

In [80]:
qubit_op.coeffs

array([ 25.924011 +0.j, -10.4946044+0.j,  -3.0924011+0.j,  -3.0924011+0.j,
         0.375    +0.j,   0.25     +0.j,   0.25     +0.j,   0.25     +0.j,
         0.25     +0.j, -10.4946044+0.j,   0.25     +0.j,   0.25     +0.j,
         0.375    +0.j])